# Landschuetzer Global Internal Variability

Draft for code to compute internal variability magnitude in CO$_{2}$ fluxes for Landschuetzer product (remapped to POP grid). This is for comparison to Fig. 2 in the paper. Will likely be run as a .py script and visualized here.

In [1]:
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline
import esmtools as et

In [2]:
filepath = '/glade/p/work/rbrady/EBUS_BGC_Variability/'
filename = 'SOM-FFN.v2016.1982-2015.POP.residuals.nc'
ds = xr.open_dataset(filepath + filename)

In [4]:
ds = ds.rename({'__xarray_dataarray_variable__': 'FG_CO2'})

# Development

In [27]:
ds = xr.open_dataset('/glade/p/work/rbrady/Landschuetzer_pCO2/' +
                     'spco2_1982-2015_MPI_SOM-FFN_v2016_on_POP' +
                     '_gx1v6.conserve.nc')
ds = ds['fgco2_raw'].to_dataset()
ds['time'] = pd.date_range('1982-01', '2016-01', freq='M')

In [28]:
# Test case to detrend, deseasonalize.
ds = ds.isel(nlat=slice(250,260), nlon=slice(150,160))

In [29]:
ds = ds['fgco2_raw'].stack(points=['nlat', 'nlon']).groupby('points') \
    .apply(et.ufunc.remove_polynomial_fit).unstack('points').to_dataset()

In [30]:
ds

<xarray.Dataset>
Dimensions:    (nlat: 10, nlon: 10, time: 408)
Coordinates:
  * time       (time) datetime64[ns] 1982-01-31 1982-02-28 1982-03-31 ...
    TLAT       (nlat, nlon) float64 18.76 18.77 18.77 18.77 18.77 18.77 ...
    TLONG      (nlat, nlon) float64 129.3 130.4 131.6 132.7 133.8 134.9 ...
  * nlat       (nlat) int64 0 1 2 3 4 5 6 7 8 9
  * nlon       (nlon) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    fgco2_raw  (time, nlat, nlon) float64 -2.187 -2.165 -2.15 -1.957 -1.664 ...

In [31]:
def remove_seasonal_cycle(ds):
    """
    Given a dataset, removes the monthly climatology.
    """
    clim = ds.groupby('time.month').mean('time')
    anom = ds.groupby('time.month') - clim
    return anom

In [34]:
ds = ds['fgco2_raw'].stack(points=['nlat','nlon']).groupby('points') \
    .apply(remove_seasonal_cycle).unstack('points')

In [35]:
ds

<xarray.DataArray 'fgco2_raw' (time: 408, nlat: 10, nlon: 10)>
array([[[-0.491918, -0.508404, ...,  0.094224,  0.022748],
        [-0.466546, -0.497008, ...,  0.380911,  0.078874],
        ...,
        [-0.250951, -0.267167, ...,  0.162747,  0.232496],
        [-0.238498, -0.258002, ...,  0.135324,  0.201829]],

       [[-0.111232, -0.028336, ..., -0.140593, -0.155437],
        [ 0.074788,  0.044526, ..., -0.07771 , -0.108465],
        ...,
        [ 0.27001 ,  0.22569 , ...,  0.317128,  0.348933],
        [ 0.2456  ,  0.204744, ...,  0.31359 ,  0.343269]],

       ...,

       [[ 0.022787,  0.025465, ..., -0.088191, -0.015069],
        [ 0.09593 ,  0.094486, ..., -0.164202, -0.109539],
        ...,
        [ 0.733964,  0.255176, ..., -0.121533, -0.229275],
        [ 0.68916 ,  0.321214, ..., -0.090577, -0.180008]],

       [[ 0.461003,  0.44955 , ...,  0.351217,  0.335444],
        [ 0.682419,  0.570593, ...,  0.531184,  0.584846],
        ...,
        [ 0.387253,  0.375784, ...,  0.5